In [ ]:
%matplotlib inline


# Auto floorplan homogeneous design

This example demonstate how to render FPGA Tile using ``FloorPlanViz`` class
User can provide external script to render tiles, by default the rendering is
based on ``initial_placement`` class.

This script can be used for shaping and placement of the modules before place and route.

<img src="file://../../../examples/OpenFPGA_basic/_fpga_auto_initial_placement.svg" width="100%" align="center">


In [ ]:
import glob
import logging

import spydrnet as sdn
from spydrnet_physical.util import OpenFPGA, initial_hetero_placement
from spydrnet_physical.util import FPGAGridGen, FloorPlanViz

logger = logging.getLogger("spydrnet_logs")
sdn.enable_file_logging(LOG_LEVEL="INFO")

STYLE_SHEET = """
    .over_util {fill:#b22222 !important}
    text{font-family: Lato; font-size: 8px}
"""

CPP = 2
SC_HEIGHT = 10


def main():
    """
    Main method
    """
    proj = "../homogeneous_fabric"
    source_files = glob.glob(f"{proj}/*_Verilog/lb/*.v")
    source_files += glob.glob(f"{proj}/*_Verilog/routing/*.v")
    source_files += glob.glob(f"{proj}/*_Verilog/sub_module/*.v")
    source_files += glob.glob(f"{proj}/*_Verilog/fpga_top.v")

    # Create OpenFPGA object
    fpga = OpenFPGA(grid=(4, 4), verilog_files=source_files)

    # Convert wires to bus structure
    fpga.annotate_area_information("./support_files/fpga_top_area.rpt")

    # Convert wires to bus structure
    fpga.create_grid_clb_bus()
    fpga.create_grid_io_bus()
    fpga.create_sb_bus()
    fpga.create_cb_bus()
    fpga.merge_all_grid_ios()

    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
    #           Floorplan visualization
    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
    fpga_grid = FPGAGridGen(
        design_name="FPGA4x4",
        layout="4x4",
        arch_file=f"{proj}/FPGA44_Task/arch/k6_N10_tileable.xml",
        release_root=None,
    )
    fpga_grid.enumerate_grid()
    fpga.load_grid(fpga_grid)
    fpga.register_placement_creator(
        initial_hetero_placement, areaFile=f"{proj}/area_info.txt"
    )
    fpga.create_placement()
    fpga.show_placement_data("*_0__*")
    fpga.design_top_stat()

    # # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
    # #           Adjust Floorplan
    # # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
    fpga.update_module_label()

    fp = FloorPlanViz(fpga.top_module)
    fp.compose(skip_connections=True, skip_pins=True)
    fp.custom_style_sheet = STYLE_SHEET
    dwg = fp.get_svg()
    dwg.add(fpga.placement_creator.design_grid.render_grid(return_group=True))

    pattern = dwg.pattern(size=(4 * CPP, 2 * SC_HEIGHT), patternUnits="userSpaceOnUse")
    pattern.add(dwg.circle(center=(2, 2), r=1, fill="black"))
    pattern.add(dwg.circle(center=(2, SC_HEIGHT + 2), r=1, fill="red"))
    dwg.defs.add(pattern)
    dwg.defs.elements[0].elements[0].attribs["fill"] = pattern.get_funciri()

    dwg.saveas("_fpga_auto_initial_placement.svg", pretty=True, indent=4)


if __name__ == "__main__":
    main()